L'idée : Mise en situation réelle , je dispose d'une base de données simple (fichier csv), je vais donc la crypter homomorphiquement et envoyer son cryptage au serveur afin de traiter la traiter et la manipuler , je peux à la fin choisir si je souhaite rècupèrer les modifications qui ont été fait par le serveur.


Dataset content  : 

id : Identifiant unique du patient.

HR_min, HR_max : Fréquence cardiaque minimale et maximale.

SBP_min, SBP_max : Pression artérielle systolique minimale et maximale.

DBP_min, DBP_max : Pression artérielle diastolique minimale et maximale.

SAPSII : Score de gravité (Simplified Acute Physiology Score II).

RR : Fréquence respiratoire.

GCS : Score de Glasgow (évaluation de l'état de conscience).

Dates : Dates d'admission à l'hôpital, en soins intensifs, de sortie ou de décès.

Données biologiques :

ddimere_J1 à J5 : D-dimères (marqueur de coagulation) sur 5 jours.

crp_J1 à J5 : Protéine C-réactive (marqueur inflammatoire) sur 5 jours.

ldh_j1 à j5 : Lactate déshydrogénase (enzyme liée aux dommages tissulaires) sur 5 jours.

L_pourc_J1 à J5 : Pourcentage de lymphocytes (marqueur immunitaire) sur 5 jours.

Autres variables :

male : Genre (1 pour masculin, 0 pour féminin).

ICU_LOS : Durée de séjour en soins intensifs.

Death_D28, Death_D7 : Indicateurs de décès à 28 jours et 7 jours.

Age : Âge du patient.

In [102]:
!pip install openpyxl


[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [114]:
from flask import Flask
import tenseal as ts
import utils

In [129]:
import pandas as pd

# Récuperrer les 3 premières colonnes du fichier Excel
medical_data = pd.read_excel('medical_dataset_01.xlsx', usecols=[0, 1, 2])


context = ts.context(ts.SCHEME_TYPE.BFV, poly_modulus_degree=8192, plain_modulus=1032193)
context.generate_galois_keys()
context.global_scale = 2**40
enc_vector_id = ts.bfv_vector(context, medical_data['id'].tolist()) 
enc_vector_HR_min = ts.bfv_vector(context, medical_data['HR_min'].tolist())
enc_vector_HR_max = ts.bfv_vector(context, medical_data['HR_max'].tolist())

my_secret_key=context.serialize(save_secret_key=True)
utils.write_data("Keys_client/secret_key.txt", my_secret_key)

context.make_context_public()
public_key=context.serialize()
utils.write_data("Keys_client/public_key.txt", public_key)


utils.write_data("outputs_client/enc_vector_id.txt", enc_vector_id.serialize())
utils.write_data("outputs_client/enc_vector_HR_min.txt", enc_vector_HR_min.serialize())
utils.write_data("outputs_client/enc_vector_HR_max.txt", enc_vector_HR_max.serialize())



In [131]:
from flask import Flask, send_from_directory, render_template, request, redirect, url_for
import os
from werkzeug.utils import secure_filename

app = Flask(__name__)

# Configuration pour l'upload
UPLOAD_FOLDER = 'uploads'
ALLOWED_EXTENSIONS = {'txt', 'pdf', 'png', 'jpg', 'jpeg', 'gif', 'enc'}
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Créer le dossier d'upload s'il n'existe pas
os.makedirs(UPLOAD_FOLDER, exist_ok=True)

def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route("/")
def bonjour():
    return render_template("index.html")

@app.route("/download/<filename>")
def download_file(filename):
    if filename == "public_key.txt":
        directory = os.path.join(os.getcwd(), "Keys_client")
    else:
        directory = os.path.join(os.getcwd(), "outputs_client")

    try:
        return send_from_directory(directory, filename, as_attachment=True)
    except FileNotFoundError:
        return "Fichier non trouvé.", 404

@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return redirect(request.url)
    
    files = request.files.getlist('file')
    
    for file in files:
        if file.filename == '':
            continue
            
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
    
    return redirect(url_for('bonjour'))

if __name__ == "__main__":
    app.run(host="0.0.0.0")
# mettre "outputs_client/enc_vector_HR_max.txt" ,  "outputs_client/enc_vector_HR_min.txt","outputs_client/enc_vector_id.txt" , et "Keys_test/public_key.txt" à télécharger depuis le localhost
   
#Conection to the server
#le client envoie les données qu'il veut , chiffrées , les définit et envoie aussi la clé publique
#Le serveur télécharge les données et la clé publique
#choisit de les manipuler 
# Moyenne (si addition homomorphique supportée)

#Somme de patients ayant une caractéristique donnée

#Fréquences d’apparition d’une pathologie

#Calculs de taux (ex. taux de réussite d’un traitement)

#Médiane ou quantiles (avec méthodes adaptées
#    
    
    
    

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.11.1.153:5000
Press CTRL+C to quit


In [ ]:
loaded_secret_key = utils.read_data("Keys_client/secret_key.txt")
secret_key = ts.context_from(loaded_secret_key).secret_key()

HR_modified_bytes = utils.read_data("outputs_server/HR_max_computed.txt")
HR_modified_vec = ts.lazy_bfv_vector_from(HR_modified_bytes)
HR_modified_vec.link_context(context)
# --- Déchiffrement du vecteur corrigé ---
HR_modified = HR_modified_vec.decrypt(secret_key)
print(" HR_max après correction machine :", HR_modified)

 HR_max après correction machine : [194.00000000005608, 204.99999999899407, 269.9999999993546, 223.99999999993568, 190.00000000050375, 195.9999999998051, 197.99999999922764, 196.00000000127088, 190.00000000032622, 195.00000000075514, 171.99999999784765, 160.99999999996703, 226.9999999989781, 193.0000000017307, 231.99999999902494, 195.99999999915946, 203.0000000009445, 234.99999999962148, 200.99999999922358, 199.00000000047052, 197.99999999999372, 203.99999999709308, 165.99999999916693, 191.99999999935213, 187.0000000005893, 206.9999999999047, 179.99999999957387, 190.99999999937742, 179.99999999987585, 172.9999999982074, 164.99999999996174, 168.0000000007043, 189.00000000029365, 193.0000000000527, 203.9999999993716, 202.99999999956773, 188.99999999959277, 193.00000000052174, 184.00000000011332, 214.99999999989564, 183.0000000005021, 179.99999999930031, 208.0000000000698, 195.0000000003456, 190.99999999910617, 208.00000000015496, 175.00000000013006, 204.99999999898205, 184.99999999976848